# Import packages

In [51]:
import requests
import json

In [68]:
url = 'https://www.brilliantearth.com/loose-diamonds/list/?shapes=Round&min_carat=0.25&max_carat=0.34&page=1'

In [69]:
response = requests.get(url).text

In [70]:
data = json.loads(response)

In [71]:
data['total_count']

15860

In [72]:
len(data['diamonds'])

20

In [13]:
diamond

{'diamonds': [{'upc': '4178464A',
   'active': True,
   'title': '0.30 Carat Round Diamond',
   'product_class': 'Loose Diamonds',
   'index_id': 10223757,
   'shape': 'Round',
   'price': 400,
   'carat': 0.3,
   'color': 'G',
   'clarity': 'SI2',
   'cut': 'Good',
   'length_width_ratio': 1,
   'collection': '',
   'report': 'GIA',
   'origin': 'Botswana Sort',
   'polish': 'Very Good',
   'symmetry': 'Very Good',
   'measurements': '4.38 x 4.34 x 2.56',
   'depth': 58.8,
   'table': 59.0,
   'girdle': 'Medium - Thick',
   'culet': 'None',
   'fluorescence': 'None',
   'shipping_day': 11,
   'color_order': 4,
   'clarity_order': 1,
   'cut_order': 3,
   'report_order': 4,
   'collection_order': None,
   'is_memo': False,
   'valid': True,
   'has_cert': True,
   'certificate_number': '6362349758',
   'shipping_supplier': 'KP Sanghvi',
   'length': 4.359999999999999,
   'inventory_location': [],
   'dedicated': False,
   'has_v360_video': False,
   'diamond_blockchain': False,
   'dia

In [ ]:
diamond.keys()

In [5]:
diamond['diamonds'][0]['upc']

'4178464A'

In [14]:
diamond['diamonds'][0]['carat']

0.3

In [15]:
diamond['diamonds'][0]['color']

'G'

In [16]:
diamond['diamonds'][0]['clarity']

'SI2'

In [17]:
diamond['diamonds'][0]['cut']

'Good'

In [18]:
diamond['diamonds'][0]['price']

400

In [127]:
class Diamonds:
    def __init__(self, page_num=1, min_carat=0.25, max_carat=0.26):
        self.page_num = page_num
        self.min_carat = min_carat
        self.max_carat = max_carat
        self.url = f'https://www.brilliantearth.com/loose-diamonds/list/?shapes=Round&min_carat={self.min_carat}&max_carat={self.max_carat}&page={self.page_num}'
        self.upc = []
        self.carat = []
        self.color = []
        self.clarity = []
        self.cut = []
        self.price = []

    
    def get_diamonds(self):

        response = requests.get(self.url).text
        data = json.loads(response)

        for diamond in data['diamonds']:
            self.upc.append(diamond['upc'])
            self.carat.append(diamond['carat'])
            self.color.append(diamond['color'])
            self.clarity.append(diamond['clarity'])
            self.cut.append(diamond['cut'])
            self.price.append(diamond['price'])
        
        if len(self.upc) < data['total_count']:
            self.page_num += 1
            self.get_diamonds()


In [128]:
bling = Diamonds()

In [129]:
bling.get_diamonds()

In [132]:
print(set(bling.upc))

{'4205714Y', '3645913A', '4226679Y', '4079843A', '3307006A', '3148398AY', '3566960A', '4205701Y', '4226681Y', '4205697Y', '4051146A', '4205700Y', '4205707Y', '4205705Y', '3981325A', '4205715Y', '3369770A', '4205719Y', '3817097A', '4226680Y'}


In [130]:
print(bling.upc)

['4051146A', '4079843A', '4205719Y', '4226679Y', '3369770A', '3148398AY', '3981325A', '3566960A', '4205697Y', '4205707Y', '3307006A', '4205705Y', '3817097A', '4226681Y', '4205715Y', '4226680Y', '4205714Y', '4205700Y', '4205701Y', '3645913A', '4051146A', '4079843A', '4205719Y', '4226679Y', '3369770A', '3148398AY', '3981325A', '3566960A', '4205697Y', '4205707Y', '3307006A', '4205705Y', '3817097A', '4226681Y', '4205715Y', '4226680Y', '4205714Y', '4205700Y', '4205701Y', '3645913A', '4051146A', '4079843A', '4205719Y', '4226679Y', '3369770A', '3148398AY', '3981325A', '3566960A', '4205697Y', '4205707Y', '3307006A', '4205705Y', '3817097A', '4226681Y', '4205715Y', '4226680Y', '4205714Y', '4205700Y', '4205701Y', '3645913A', '4051146A', '4079843A', '4205719Y', '4226679Y', '3369770A', '3148398AY', '3981325A', '3566960A', '4205697Y', '4205707Y', '3307006A', '4205705Y', '3817097A', '4226681Y', '4205715Y', '4226680Y', '4205714Y', '4205700Y', '4205701Y', '3645913A', '4051146A', '4079843A', '4205719Y',

In [103]:
import pandas

In [105]:
len(set(bling.upc))

20

In [77]:
print(len(upc))
print(carat)
print(color)
print(clarity)
print(cut)
print(price)

20
[0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.31, 0.31, 0.32]
['G', 'I', 'J', 'I', 'G', 'I', 'G', 'I', 'G', 'F', 'J', 'J', 'I', 'H', 'I', 'J', 'G', 'G', 'J', 'I']
['SI2', 'SI2', 'SI1', 'SI1', 'SI2', 'SI1', 'SI2', 'SI2', 'SI2', 'SI2', 'SI1', 'SI1', 'SI1', 'SI1', 'SI1', 'VS2', 'SI2', 'SI2', 'SI1', 'SI2']
['Good', 'Ideal', 'Very Good', 'Very Good', 'Very Good', 'Very Good', 'Very Good', 'Very Good', 'Very Good', 'Very Good', 'Ideal', 'Super Ideal', 'Very Good', 'Good', 'Ideal', 'Very Good', 'Very Good', 'Very Good', 'Super Ideal', 'Super Ideal']
[400, 430, 430, 430, 440, 440, 440, 440, 440, 440, 440, 440, 440, 440, 440, 440, 450, 450, 450, 450]


In [49]:
get_diamonds()

In [47]:
print(upc)
print(carat)
print(color)
print(clarity)
print(cut)
print(price)

['4142813A', '4125359A', '4143034A', '4142928A', '4170271A', '4125337A', '4210004A', '4209950A', '4051146A', '3950389A', '3736066A', '4142844A', '4125345A', '4142908A', '3981439A', '3950371A', '4143064A']
[0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.32, 0.3, 0.25, 0.3, 0.3, 0.3, 0.3, 0.3, 0.31, 0.3, 0.3]
['I', 'G', 'H', 'H', 'I', 'G', 'J', 'J', 'F', 'F', 'F', 'J', 'J', 'J', 'F', 'F', 'J']
['SI1', 'SI2', 'SI2', 'SI2', 'SI1', 'SI2', 'SI2', 'SI1', 'SI1', 'SI2', 'SI2', 'SI1', 'SI2', 'VVS2', 'SI2', 'SI2', 'VS1']
['Super Ideal', 'Ideal', 'Very Good', 'Ideal', 'Ideal', 'Very Good', 'Super Ideal', 'Very Good', 'Super Ideal', 'Ideal', 'Very Good', 'Super Ideal', 'Super Ideal', 'Very Good', 'Ideal', 'Very Good', 'Very Good']
[450, 450, 450, 450, 450, 450, 450, 450, 450, 450, 450, 450, 450, 460, 460, 460, 460]


In [50]:
print(upc)
print(carat)
print(color)
print(clarity)
print(cut)
print(price)

['4178464A', '4143017A', '4143075A', '4142792A', '4142941A', '4142940A', '4142815A', '4143056A', '4079950A', '3905242A', '4142922A', '4142991A', '4143050A', '4041757A', '4143067A', '4142916A', '4142910A']
[0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3]
['G', 'I', 'J', 'I', 'G', 'G', 'I', 'I', 'G', 'F', 'J', 'J', 'I', 'H', 'I', 'J', 'G']
['SI2', 'SI2', 'SI1', 'SI1', 'SI2', 'SI2', 'SI1', 'SI2', 'SI2', 'SI2', 'SI1', 'SI1', 'SI1', 'SI1', 'SI1', 'VS2', 'SI2']
['Good', 'Ideal', 'Very Good', 'Very Good', 'Very Good', 'Very Good', 'Very Good', 'Very Good', 'Very Good', 'Very Good', 'Ideal', 'Super Ideal', 'Very Good', 'Good', 'Ideal', 'Very Good', 'Very Good']
[400, 430, 430, 430, 440, 440, 440, 440, 440, 440, 440, 440, 440, 440, 440, 440, 450]
